In [1]:
TEST_PATH = '../input/Raw Text/FAQ/faq.json'
LEGIS_PATH = '../input/Raw Text/Legislation/output_legislation_2.json'
NORMS_PATH = '../input/Raw Text/Norms/output_prof3_v11_2.json'

In [2]:
import json

test_set = json.load(open(TEST_PATH, 'r'))
legis_set = json.load(open(LEGIS_PATH, 'r'))
norms_set = json.load(open(NORMS_PATH, 'r'))

context_set = {**norms_set, **legis_set}

In [3]:
perg_resposta_esperada = []

for i in range(len(test_set)):
  pr = test_set[i]

  pergunta = pr['question']
  resposta = pr['answer']
  contexto = pr['context']
  
 
  perg_resposta_esperada.append({"pergunta": pergunta, "resposta": resposta, "contexto": contexto})

  if i < 3:
    print(f"{i}: {pergunta}\n{resposta}\n{contexto}")
    print('----------------------------------------------------------------------')

0: Em quais hipóteses a Procuradoria Geral pode exercer a representação judicial e extrajudicial de servidores da UNICAMP?
A Procuradoria Geral da UNICAMP integra a Advocacia Pública e está vinculada à Procuradoria Geral do Estado, para fins de atuação uniforme e coordenada, nos termos do artigo 101 da Constituição Estadual. Além disso, é o órgão de representação jurídica da Universidade e de assessoramento jurídico da Reitoria, consoante prevê o artigo 95 do Regimento Geral da UNICAMP. Desta forma, conforme previsão regimental, a Procuradoria Geral tem a função institucional de defender os interesses da universidade, enquanto autarquia estadual, estritamente vinculada ao atendimento do interesse público. A representação judicial ou extrajudicial de servidor com vínculo funcional permanente poderá ocorrer em caráter excepcional, nas hipóteses em que o ato impugnado decorrer do exercício de suas atribuições constitucionais, legais ou regulamentares, quando o ato praticado tiver seguido 

In [4]:
all_titles = []
documents = []

for item in test_set:
    for context in item['context']:
        if context.lower() not in all_titles:
            all_titles.append(context.lower())
            documents.append({
                    "title": context,
                    "content": context_set[context] if context in context_set else item['answer']
                }
            )

In [5]:
%%time
from tqdm import tqdm
import spacy
import pickle

nlp = spacy.blank("pt")
nlp.add_pipe("sentencizer")

stride = 2
max_length = 3

def window(documents, stride=2, max_length=3):
  treated_documents = []

  for j, document in enumerate(tqdm(documents)):
    doc_text = document['content']
    doc = nlp(doc_text)
    sentences = [sent.text.strip() for sent in doc.sents]
    for i in range(0, len(sentences), stride):
      segment = ' '.join(sentences[i:i + max_length])
      treated_documents.append({
          "title": document['title'],
          "contents": "[" + document['title'] + "]" + ": " + segment,
          "segment": segment
      })
      if i + max_length >= len(sentences):
        break
  return treated_documents


artigos_segmentados = window(documents)

100%|██████████| 38/38 [00:02<00:00, 18.73it/s]

CPU times: user 4.59 s, sys: 328 ms, total: 4.92 s
Wall time: 3.89 s


In [6]:
%%time
from collections import Counter
import array
import pickle
import math
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

nltk.download('punkt')
nltk.download('stopwords')

# Definição de uma classe para índice invertido
class IndiceInvertido:

  # Recebe 'tokenizar', uma função tokenizadora
  def __init__(self):
    # Cria um índice invertido vazio
    self.indice = {}
    # Cria um índice de tamanho de documentos vazio
    self.tamanho_doc = {}
    # Guarda o total de documentos adicionados
    self.n_docs = 0
    # Tokenizador
    self.stemmer = PorterStemmer()

  def tokenizar(self, texto):
    # Tokeniza
    tokens = word_tokenize(texto)
    # Converte para minúsculo, filtra stopwords e passa pelo Porter Stemer
    tokens = [self.stemmer.stem(token.lower()) for token in tokens if token.lower() not in stopwords.words('portuguese')] # changed to portuguese

    return tokens

  def adiciona_doc(self, id_doc, conteudo_doc=None):
    tokens = self.tokenizar(conteudo_doc)

    contador_tokens_do_documento = Counter(tokens)
    for token, n_ocorrencias in contador_tokens_do_documento.items():
      self.indice.setdefault(token, {"id_doc": [], "n_ocorrencias": array.array("L", [])})['id_doc'].append(id_doc)
      self.indice.setdefault(token, {"id_doc": [], "n_ocorrencias": array.array("L", [])})['n_ocorrencias'].append(n_ocorrencias)

    self.n_docs += 1
    self.tamanho_doc[id_doc] = len(tokens)

class BM25:

  def __init__(self, indiceInvertido=IndiceInvertido(), k1 = 0.9, b = 0.4, bias_adicionar_ao_idf = 0):
    self.indiceInvertido = indiceInvertido
    self.bias_adicionar_ao_idf = bias_adicionar_ao_idf
    self.calcula_tam_medio_doc_no_indice()
    self.k1 = k1
    self.b = b
    self.precalcula_idf()
    self.reinicia_score_dos_indices()

  def reinicia_score_dos_indices(self):
    for token in self.indiceInvertido.indice.keys():
      self.indiceInvertido.indice[token].pop('score', None)

  def calcula_tam_medio_doc_no_indice(self):
    self.avgdl = sum(self.indiceInvertido.tamanho_doc.values()) / self.indiceInvertido.n_docs

  def precalcula_idf(self):
    # Número de documento do corpus está presente no objeto indiceInvertido
    N = self.indiceInvertido.n_docs
    # Varre todos os tokens do índice. Os tokens são as chaves do indiceInvertido.indice
    for token in self.indiceInvertido.indice.keys():
      # O número de documentos que possui o token é calculado pelo tamanho da lista de id_doc:
      n_doc_token = len(self.indiceInvertido.indice[token]['id_doc'])
      # Isso já é o suficiente pra calcular o idf
      idf_token = math.log( ((self.indiceInvertido.n_docs - n_doc_token + 0.5)/(n_doc_token + 0.5)) + self.bias_adicionar_ao_idf )
      # E agora, vamos colocar essa informação no índice
      self.indiceInvertido.indice[token]['idf'] = idf_token

  def calcula_score_para_um_token_e_salva(self, token):
    # O cálculo do BM25 para determinada query é a multiplicação do idf pela frequência do termo no documento * (k1 + 1)
    # Além disso, é dividido pela frequencia do termo no documento + k1 * (1 - b + b * tamanho_doc/avgdl)
    idf = self.indiceInvertido.indice[token]['idf']
    # Juntando tudo, podemos calcular o score pelo BM25
    zip_id_freq = zip(self.indiceInvertido.indice[token]['id_doc'], self.indiceInvertido.indice[token]['n_ocorrencias'])
    bm25 = array.array("f", [ idf * freq_token_no_doc * (self.k1 + 1) / (freq_token_no_doc + self.k1 * (1 - self.b + self.b * self.indiceInvertido.tamanho_doc[id_doc] / self.avgdl)) for (id_doc, freq_token_no_doc) in zip_id_freq ])
    # Salva o bm25 no índice
    self.indiceInvertido.indice[token]['score'] = bm25

  def tokenizar(self, query):
    return self.indiceInvertido.tokenizar(query)

  def pesquisar(self, query):
    # Tokeniza a query
    tokens = self.tokenizar(query)

    # Se não tem token para ser pesquisado, retorna conjunto vazio
    if (len(tokens) == 0):
      return []

    # Guarda um dicionário onde a chave é o id do documento e o valor é o score desse documento para a query pesquisada
    docs_retornado_com_score = Counter({})

    # Faz a pesquisa de documentos. Para isso iteramos todos os tokens da query
    for token in tokens:
      # É possível que a query contenha algum termo que não foi indexado. Se isso ocorrer,
      # entende-se que a frequência desse token em qualquer documento é 0, já que não pode ser encontrado
      if token not in self.indiceInvertido.indice:
        continue

      # Pega a lista de documentos que será analisado
      docs_que_tem_token = self.indiceInvertido.indice[token]['id_doc']

      # Se for a primeira vez que esse token é pesquisado, é necessário calcular o score relacionado
      # a ele e salvar. Se já tiver sido feito antes, já podemos buscar o cálculo pronto (que funciona
      # como um cache. Isso é útil no caso de várias pesquisas seguidas)
      if 'score' not in self.indiceInvertido.indice[token].keys():
        self.calcula_score_para_um_token_e_salva(token)
      score_dos_docs_deste_token = self.indiceInvertido.indice[token]['score']

      # Agora já temos calculado o score de todos os documentos desse token. Só adiciona ao acumulador de score atual
      # docs_retornado_com_score += score_dos_docs_deste_token -> Se fosse usar dict direto no índice seria assim, mas a memória não está aguentando guardar os scores de ambos
      for id_doc, score_par_doc_token in zip(docs_que_tem_token, score_dos_docs_deste_token):
        docs_retornado_com_score[id_doc] += score_par_doc_token

    # Agora converte esse dict em uma lista de tuplas com a chave (id_doc) e valor (score_do_doc)
    docs_com_score = list(docs_retornado_com_score.items())

    # E ordena do mais relevante para o menos relevante
    return sorted(docs_com_score, key=lambda x: x[1], reverse=True)

CPU times: user 1.71 s, sys: 203 ms, total: 1.92 s
Wall time: 1.73 s


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/victorgmoreno/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/victorgmoreno/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
iidx = IndiceInvertido()
iidx.tokenizar('Qual o maior planeta do sistema solar?')

['maior', 'planeta', 'sistema', 'solar', '?']

In [8]:
# Se True, gera o índice invertido e salva em um pickle. Se False, apenas recupera o pickle do drive
GERAR_INDICE_INVERTIDO = True
ARQUIVO_INDICE_INVERTIDO = 'iidx_artigos_segmentados_6' # A função salva em 2 arquivos, então ela mesma coloca a extensão pickle

# Se True, gera as respostas. Se False, apenas recupera o pickle do drive
AGREGAR_DOCUMENTOS = True
ARQUIVO_AGREGADO = 'respostas_obtidas.pickle_9'

In [9]:
import pickle

def salvar_indice_em_arquivo_pickle(indice_invertido):
  nome_arquivo_base = f"{'../input/Raw Text/Rag Data/'}{ARQUIVO_INDICE_INVERTIDO}"

  with open(f'{nome_arquivo_base}_indice.pickle', 'wb') as f:
    pickle.dump(indice_invertido.indice, f)
  with open(f'{nome_arquivo_base}_tamanho_doc.pickle', 'wb') as f:
    pickle.dump(indice_invertido.tamanho_doc, f)

def recuperar_indice_em_arquivo_pickle():
  nome_arquivo_base = f"{'../input/Raw Text/Rag Data/'}{ARQUIVO_INDICE_INVERTIDO}"
  idx = IndiceInvertido()

  with open(f'{nome_arquivo_base}_indice.pickle', 'rb') as f:
    idx.indice = pickle.load(f)
  with open(f'{nome_arquivo_base}_tamanho_doc.pickle', 'rb') as f:
    idx.tamanho_doc = pickle.load(f)
  idx.n_docs = len(idx.tamanho_doc)

  return idx

In [10]:
%%time
iidx = IndiceInvertido()

if GERAR_INDICE_INVERTIDO:
  # Vamos usar o próprio índice como um id para o segmento. Vai facilitar a vida depois
  # art_seg['contents'] tem o título e 3 frases do artigo
  for id, art_seg in enumerate(artigos_segmentados):
    iidx.adiciona_doc(id, art_seg['contents'])

    if id % 10000 == 0:
      print(f'{id} segmentos indexados')

  salvar_indice_em_arquivo_pickle(iidx)
else:
  iidx = recuperar_indice_em_arquivo_pickle()

0 segmentos indexados
CPU times: user 17.2 s, sys: 3.86 s, total: 21 s
Wall time: 22.7 s


In [11]:
buscador = BM25(iidx, 0.82, 0.68, 1)
buscador = BM25(iidx)

In [12]:
print('Pergunta e resposta esperada: ')
print(perg_resposta_esperada[0])

resultado_bm25 = buscador.pesquisar(perg_resposta_esperada[0]['pergunta'])

print('-----------------------------------------------------------------')
print('10 primeiros conteúdos retornados pelo BM25\n')
for i in range(10):
  segmento_id = resultado_bm25[i][0]
  score = resultado_bm25[i][1]

  title = artigos_segmentados[segmento_id]['title']
  contents = artigos_segmentados[segmento_id]['contents']
  segment = artigos_segmentados[segmento_id]['segment']

  print(f'Segmento_id: {segmento_id}. Score: {score}')
  print(f"Title: {title}")
  print(f"Contents: {contents}")
  print('-----------------------------------------------------------------')

Pergunta e resposta esperada: 
{'pergunta': 'Em quais hipóteses a Procuradoria Geral pode exercer a representação judicial e extrajudicial de servidores da UNICAMP?', 'resposta': 'A Procuradoria Geral da UNICAMP integra a Advocacia Pública e está vinculada à Procuradoria Geral do Estado, para fins de atuação uniforme e coordenada, nos termos do artigo 101 da Constituição Estadual. Além disso, é o órgão de representação jurídica da Universidade e de assessoramento jurídico da Reitoria, consoante prevê o artigo 95 do Regimento Geral da UNICAMP. Desta forma, conforme previsão regimental, a Procuradoria Geral tem a função institucional de defender os interesses da universidade, enquanto autarquia estadual, estritamente vinculada ao atendimento do interesse público. A representação judicial ou extrajudicial de servidor com vínculo funcional permanente poderá ocorrer em caráter excepcional, nas hipóteses em que o ato impugnado decorrer do exercício de suas atribuições constitucionais, legais

In [13]:
artigos_segmentados

[{'title': 'Constituição Estadual SP',
  'contents': '[Constituição Estadual SP]: A\nFicha informativa\nTexto com alterações\nCONSTITUIÇÃO EST ADUAL, DE 05 DE OUTUBRO DE 1989\n(Última atualização: Emenda Constitucional n° 53, de 19/12/2023)\nPREÂMBULO\nO Povo Paulista, invocando a proteção de Deus, e inspirado nos princíp ios constitucionais da República e no ideal de a todos assegurar justiça\ne bem-estar , decreta e promulga, por seus representantes, a CONSTITUIÇÃO DO EST ADO DE SÃO P AULO. TÍTULO I\nDos Fundamentos do Estado\nArtigo 1° - O Estado de São Paulo, integrant e da República Federativa do Brasil, exerce as competências que não lhe são vedadas pela\nConstituição Federal. Artigo 2° - A lei estabelecerá procedimentos judiciários abreviados e de custos reduzidos para as ações cujo objeto principal seja a salvaguarda\ndos direitos e liberdades fundamentais.',
  'segment': 'A\nFicha informativa\nTexto com alterações\nCONSTITUIÇÃO EST ADUAL, DE 05 DE OUTUBRO DE 1989\n(Última atua

In [14]:
initial_prompt = """
Você é um assistente digital especializado em regulamentações, normas e legislação da Unicamp. Seu papel é utilizar uma biblioteca digital de documentos oficiais (como resoluções, estatutos e diretrizes) para responder precisamente a perguntas relacionadas a esses documentos. Suas respostas devem ser estritamente baseadas nas informações contidas nos documentos e não devem incluir conhecimento prévio ou externo. Além disso, responda em PT BR.

Ao responder a uma pergunta, siga estes passos:

1. **Identificação do Documento**: Determine qual(is) documento(s) da biblioteca são mais relevantes para a pergunta. Cite o nome do documento entre colchetes, por exemplo, [Resolução GR-018/2018].

2. **Extrato Relevante**: Localize e transcreva o trecho específico do documento que diretamente responde ou esclarece a pergunta. Se necessário, inclua múltiplos trechos para cobrir todos os aspectos da pergunta.

3. **Explicação do Raciocínio**: Explique claramente como os trechos citados apoiam sua resposta. Esta explicação deve conectar os pontos dos documentos à pergunta, esclarecendo qualquer interpretação ou inferência que foi necessária.

4. **Resposta Concisa**: Forneça uma resposta direta à pergunta, limitada a no máximo 30 palavras. Esta resposta deve ser uma síntese do que foi explicado anteriormente.

5. **Considerações a serem feitas para responder uma pergunta:**:
   - **Relevância**: Verifique se a resposta aborda todos os aspectos da pergunta.
   - **Fidelidade**: Assegure que a resposta esteja fielmente baseada nos documentos.
   - **Recall Contextual**: Avalie se todos os contextos relevantes foram considerados na resposta.
   - **Precisão Contextual**: Garanta que os trechos citados são diretamente relevantes para a pergunta.

Exemplo de pergunta: 'Qual o procedimento para a submissão de projetos de pesquisa na Unicamp que envolvam parcerias internacionais?'
Exemplo de resposta: Evidence: [Deliberação CAD-A-009/2023]: Esta deliberação estabelece os critérios quantitativos mínimos para progressão na Carreira de Pesquisador na Faculdade de Ciências Médicas. Para progressão de Pq-C para Pq-B, o pesquisador deve alcançar 190 pontos, incluindo um mínimo de 18 pontos em projetos de pesquisa coordenados e 70 pontos em atividades de pesquisa de alto impacto. Para a progressão de Pq-B para Pq-A, são necessários 380 pontos, com um mínimo de 36 pontos em projetos de pesquisa coordenados e 120 pontos em pesquisas de alto impacto. Answer: '[Deliberação CAD-A-009/2023]: A Deliberação CAD-A-009/2023 da Universidade Estadual de Campinas define o Perfil Quantitativo Mínimo para a ascensão aos níveis superiores da Carreira de Pesquisador (Pq) na Faculdade de Ciências Médicas. Para avançar do nível Pq-C para Pq-B, um pesquisador deve alcançar um total de 190 pontos, distribuídos entre projetos de pesquisa e outras atividades científicas. Para a progressão do nível Pq-B para Pq-A, o pesquisador deve somar 380 pontos, demonstrando maior liderança e impacto em sua área. Estes pontos são acumulados a partir de projetos de pesquisa, publicações, e contribuições a atividades acadêmicas e administrativas, conforme detalhado nos critérios específicos de pontuação. 'Evidence: '
"""

In [15]:
def mensagem_user(numExemplo, pergunta, lista_documentos):
  doc_com_id = [ f"{lista_documentos[i]}" for i in range(len(lista_documentos))]
  return f"Example {numExemplo}\n\n" + "\n\n".join(doc_com_id) + f"\n\nQuestion: {pergunta}"


def mensagem_assistant(evidencia, resposta):
  return f"Evidence: {evidencia}\n\nAnswer: {resposta}"


few_shot_examples = [
    {"user": "", "assistant": ""},
    {"user": "", "assistant": ""},
]
few_shot_examples[0]["user"] = mensagem_user(1, "Pink Floyd tem uma música sobre a Riviera Francesa?", [
    "[Nome do documento 1]: \"San Tropez\" é a quarta faixa do álbum Meddle da banda Pink Floyd. Esta música foi uma das várias consideradas para o álbum de melhores sucessos da banda, Echoes: The Best of Pink Floyd.",
    "[Nome do documento 2]: A Riviera Francesa (conhecida em francês como Côte d'Azur [kot daˈzyʁ]; Occitano: Còsta d'Azur [ˈkɔstɔ daˈzyɾ]; tradução literal \"Costa Azul\") é a costa do Mediterrâneo no sudeste da França. Não há um limite oficial, mas geralmente considera-se que se estende de Cassis, Toulon ou Saint-Tropez a oeste até Menton na fronteira França-Itália a leste, onde se junta a Riviera Italiana. A costa está inteiramente dentro da região de Provence-Alpes-Côte d'Azur (Região Sul) da França. O Principado de Mônaco é uma semi-enclave dentro da região, cercado por três lados pela França e de frente para o Mediterrâneo.",
    "[Nome do documento 3]: Moon também prometeu transparência em sua presidência, mudando a residência presidencial da palaciana e isolada Casa Azul para um complexo governamental existente no centro de Seul.",
    "[Nome do documento 4]: Saint-Tropez (EUA: /ˌsæn troʊˈpeɪ/ SAN-troh-PAY, francês: [sɛ̃ tʁɔpe]; Occitano: Sant-Tropetz, pronunciado [san(t) tʀuˈpes]) é uma cidade na Riviera Francesa, 68 quilômetros (42 milhas) a oeste de Nice e 100 quilômetros (62 milhas) a leste de Marselha no departamento de Var da região de Provence-Alpes-Côte d'Azur de Occitania, sul da França."
])
few_shot_examples[0]["assistant"] = mensagem_assistant("\"San Tropez\" é uma música do Pink Floyd sobre a Riviera Francesa, de acordo com o trecho '\"San Tropez\" é a quarta faixa do álbum Meddle da banda Pink Floyd' em [Nome do documento 1]. Saint-Tropez é uma cidade na Riviera Francesa, de acordo com o trecho 'Saint-Tropez (EUA: /ˌsæn troʊˈpeɪ/ SAN-troh-PAY, francês: [sɛ̃ tʁɔpe]; Occitano: Sant-Tropetz, pronunciado [san(t) tʀuˈpes]) é uma cidade na Riviera Francesa' de [Nome do documento 4]", "sim")
few_shot_examples[1]["user"] = mensagem_user(2, "Em que data ocorreu o terremoto de Loma Prieta?", [
    "[Nome do documento 1]: O terremoto de Loma Prieta de 1989 ocorreu na Costa Central da Califórnia em 17 de outubro às hora local (1989-10-18 00:04 UTC). O choque foi centrado no Parque Estadual da Floresta de Nisene Marks aproximadamente 10 mi a nordeste de Santa Cruz em uma seção do Sistema de Falhas de San Andreas e foi nomeado após o pico Loma Prieta nas Montanhas de Santa Cruz. Com uma magnitude de 6.9 e uma intensidade máxima de Mercalli Modificado de IX (Violento), o choque foi responsável por 63 mortes e 3.757 feridos.",
    "[Nome do documento 2]: Foreshocks. Os eventos de 5.3 de junho de 1988 e de 5.4 de agosto de 1989 também ocorreram em falhas reversas oblíquas anteriormente desconhecidas e estavam dentro de 3 mi do epicentro do principal abalo de Loma Prieta M6.9, perto da interseção das falhas de San Andreas e Sargent. O deslocamento total para esses choques foi relativamente pequeno (aproximadamente 4 in de deslizamento lateral e substancialmente menos deslizamento reverso) e embora tenham ocorrido em falhas separadas e bem antes do abalo principal, um grupo de sismólogos considerou esses como precursores devido à sua localização no espaço e tempo em relação ao evento principal.",
    "[Nome do documento 3]: O choque de 27 de junio de 1988 ocorreu com uma intensidade máxima de VI (Forte). Seus efeitos incluíram janelas quebradas em Los Gatos e outros danos leves em Holy City, onde foi observado um aumento de fluxo em um poço de água. Mais longe das Montanhas de Santa Cruz, pedaços de concreto caíram de uma estrutura de estacionamento no Sunnyvale Town Center, um shopping de dois níveis no condado de Santa Clara.",
    "[Nome do documento 4]: O terremoto de Loma Prieta de 6.9 ocorreu em 17 de outubro de 1989. A ruptura estava relacionada ao sistema de falhas de San Andreas e afetou toda a área da Baía de San Francisco com uma intensidade máxima de Mercalli de IX (Violento). Muitas estruturas em Oakland foram gravemente danificadas, incluindo a porção de dois andares da Interestadual 880 que desabou.",
])
few_shot_examples[1]["assistant"] = mensagem_assistant("De acordo com o trecho 'O terremoto de Loma Prieta de 1989 ocorreu na Costa Central da Califórnia em 17 de outubro às hora local (1989-10-18 00:04 UTC)' de [Nome do documento 1] e o trecho 'O terremoto de Loma Prieta de 6.9 ocorreu em 17 de outubro de 1989' de [Nome do documento 4], o terremoto de Loma Prieta ocorreu em 17 de outubro de 1989", "1989-10-17")


initial_messages=[
    {"role": "system", "content": initial_prompt},
    {"role": "user", "content": few_shot_examples[0]["user"]},
    {"role": "assistant", "content": few_shot_examples[0]["assistant"]},
    {"role": "user", "content": few_shot_examples[1]["user"]},
    {"role": "assistant", "content": few_shot_examples[1]["assistant"]}
]

In [16]:
def get_passagens(pergunta, n=5):
  passagens = []
  resultado_bm25 = buscador.pesquisar(pergunta)

  for i in range(min(len(resultado_bm25), n)):
    segmento_id = resultado_bm25[i][0]
    segment = artigos_segmentados[segmento_id]['contents'] # changed segment to contents
    passagens.append(segment)

  return passagens


In [17]:
# Teste de como chamar o mensagem_user usando o retorno do BM25:

teste_pergunta = test_set[0]['question']
teste_passagens = get_passagens(teste_pergunta, 5)
print(mensagem_user(2, teste_pergunta, teste_passagens))

Example 2

[Constituição Federal]: (Incluído pela Emenda Constitucional nº 45, de 2004)
  Art. 129. São funções institucionais do Ministério Público:07/06/2024, 11:39 Constituição
www.planalto.gov.br/ccivil_03/constituicao/constituicao.htm 91/226
I - promover , privativamente, a ação penal pública, na forma da lei;
II - zelar pelo efetivo respeito dos Poderes Públicos e dos serviços de relevância pública aos direitos assegurados nesta Constituição, promovendo as medidas necessárias
a sua garantia;
III - promover o inquérito civil e a ação civil pública, para a proteção do patrimônio público e social, do meio ambiente e de outros interesses difusos e coletivos;
IV - promover a ação de inconstitucionalidade ou representação para fins de intervenção da União e dos Estados, nos casos previstos nesta Constituição;
V - defender judicialmente os direitos e interesses das populações indígenas;
VI - expedir notificações nos procedimentos administrativos de sua competência, requisitando informa

In [18]:
teste_passagens

['[Constituição Federal]: (Incluído pela Emenda Constitucional nº 45, de 2004)\n\uf24e  Art. 129. São funções institucionais do Ministério Público:07/06/2024, 11:39 Constituição\nwww.planalto.gov.br/ccivil_03/constituicao/constituicao.htm 91/226\nI - promover , privativamente, a ação penal pública, na forma da lei;\nII - zelar pelo efetivo respeito dos Poderes Públicos e dos serviços de relevância pública aos direitos assegurados nesta Constituição, promovendo as medidas necessárias\na sua garantia;\nIII - promover o inquérito civil e a ação civil pública, para a proteção do patrimônio público e social, do meio ambiente e de outros interesses difusos e coletivos;\nIV - promover a ação de inconstitucionalidade ou representação para fins de intervenção da União e dos Estados, nos casos previstos nesta Constituição;\nV - defender judicialmente os direitos e interesses das populações indígenas;\nVI - expedir notificações nos procedimentos administrativos de sua competência, requisitando in

In [19]:
from groq import Groq

client = Groq(api_key='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')

def chat_completion(pergunta, n=5):
    # Documentação: https://console.groq.com/docs/text-chat
    messages = []
    # Adiciona as mensagens de sistema e few-shot
    messages.extend(initial_messages)
    # Sempre adiciona o prompt do usuário
    passagens = get_passagens(pergunta, n)
    user_message = mensagem_user(3, pergunta, passagens)
    messages.append({
        # Set a user message for the assistant to respond to.
        "role": "user",
        "content": user_message
    })
    chat_completion = client.chat.completions.create(
        messages=messages,

        # The language model which will generate the completion.
        model="llama3-70b-8192",

        # Controls randomness: lowering results in less random completions.
        # As the temperature approaches zero, the model will become deterministic
        # and repetitive.
        temperature=0,

        # The maximum number of tokens to generate. Requests can use up to
        # 32,768 tokens shared between prompt and completion.
        max_tokens=1024,

        # Controls diversity via nucleus sampling: 0.5 means half of all
        # likelihood-weighted options are considered.
        top_p=1,

        # A stop sequence is a predefined or user-specified text string that
        # signals an AI to stop generating content, ensuring its responses
        # remain focused and concise. Examples include punctuation marks and
        # markers like "[end]".
        stop=None,

        # If set, partial message deltas will be sent.
        stream=False,
    )
    # Retorna o texto da primeira mensagem e, caso seja necessário usar depois,
    # todo o objeto
    return chat_completion.choices[0].message.content, chat_completion

In [20]:
perg_resp_esperada_obtida = []

In [21]:
from tqdm import tqdm

def salvar_perg_resp_esperada_obtida(perg_resp_esperada_obtida):
  nome_arquivo = f"{'../input/Raw Text/Rag Data/'}{ARQUIVO_AGREGADO}"
  with open(nome_arquivo, 'wb') as f:
    pickle.dump(perg_resp_esperada_obtida, f)

def recuperar_perg_resp_esperada_obtida():
  nome_arquivo_base = f"{'../input/Raw Text/Rag Data/'}{ARQUIVO_AGREGADO}"
  perg_respostas = []
  with open(nome_arquivo_base, 'rb') as f:
    perg_respostas = pickle.load(f)
  return perg_respostas

In [22]:
import time

# Do jeito que foi implementado, se der problema (rate limit), basta executar a
# célula novamente. Ele vai continuar de onde parou
if AGREGAR_DOCUMENTOS:
  for perg_resp in tqdm(perg_resposta_esperada):
    pergunta = perg_resp['pergunta']
    resposta_esperada = perg_resp['resposta']
    resposta_obtida = chat_completion(pergunta)

    perg_resp_esperada_obtida.append({
        "pergunta": pergunta,
        "resposta_esperada": resposta_esperada,
        "resposta_obtida": resposta_obtida
    })
    time.sleep(5)
  salvar_perg_resp_esperada_obtida(perg_resp_esperada_obtida)
else:
  perg_resp_esperada_obtida = recuperar_perg_resp_esperada_obtida()

100%|██████████| 50/50 [32:12<00:00, 38.66s/it]


In [23]:
salvar_perg_resp_esperada_obtida(perg_resp_esperada_obtida)

In [24]:
# Funções normize_answer e token_f1_score foram retiradas do Visconde (arquivo qasper_evaluator.py)
import re
import string

def normalize_answer(s):
  """
  Taken from the official evaluation script for v1.1 of the SQuAD dataset.
  Lower text and remove punctuation, articles and extra whitespace.
  """

  def remove_articles(text):
      return re.sub(r"\b(um|uma|o|a)\b", " ", text)

  def white_space_fix(text):
      return " ".join(text.split())

  def remove_punc(text):
      exclude = set(string.punctuation)
      return "".join(ch for ch in text if ch not in exclude)

  def lower(text):
      return text.lower()

  return white_space_fix(remove_articles(remove_punc(lower(s))))

def token_f1_score(prediction, ground_truth):
  """
  Taken from the official evaluation script for v1.1 of the SQuAD dataset.
  """
  prediction_tokens = normalize_answer(prediction).split()
  ground_truth_tokens = normalize_answer(ground_truth).split()
  common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
  num_same = sum(common.values())
  if num_same == 0:
      return 0
  precision = 1.0 * num_same / len(prediction_tokens)
  recall = 1.0 * num_same / len(ground_truth_tokens)
  f1 = (2 * precision * recall) / (precision + recall)
  return f1

def extrai_resposta(resp):
  idx_resposta_llm = resp.find('Answer: ') + 8
  return resp[idx_resposta_llm:]


# Trata respostas vazias do GPT-3.5
def uniformiza_resposta_vazia(resp):
  valores_possiveis = ["n/a", "it is not possible to determine", "no information in the documents", "no information provided", "none of the documents", "documents do not provide", "unknown", "cannot be answered", "not mentioned", "not specified", "n/a", "not available", "insufficient data", "not enough information", "cannot be determined", "there is no mention", "no information available"]

  if resp == '':
    return 'none'

  resp = resp.lower()

  for valor_none in valores_possiveis:
    if valor_none in resp:
      return 'none'

  return resp

In [25]:
import pandas as pd

col_queries = []
col_resp_esperadas = []
col_respostas_llm = []
col_f1_score = []
col_em_score = []

for perg_respostas in perg_resp_esperada_obtida:
  query = perg_respostas['pergunta']
  resposta_esperada = normalize_answer(perg_respostas['resposta_esperada'])
  resposta_llm = normalize_answer(uniformiza_resposta_vazia(extrai_resposta(perg_respostas['resposta_obtida'][0])))
  #resposta_llm_2 = normalize_answer(uniformiza_resposta_vazia(perg_respostas['resposta_obtida']))
  #print(perg_respostas['resposta_obtida'])
  f1_score = token_f1_score(resposta_llm, resposta_esperada)
  em_score = 1 if resposta_llm == resposta_esperada else 0


  col_queries.append(query)
  col_resp_esperadas.append(resposta_esperada)
  col_respostas_llm.append(resposta_llm)
  col_f1_score.append(f1_score)
  col_em_score.append(em_score)

df = pd.DataFrame({'Query': col_queries, 'Resp_esperada': col_resp_esperadas, 'Resposta_LLM': col_respostas_llm, 'F1_score': col_f1_score, 'EM_score': col_em_score})
df

,Query,Resp_esperada,Resposta_LLM,F1_score,EM_score
0,Em quais hipóteses a Procuradoria Geral pode e...,procuradoria geral da unicamp integra advocaci...,não há resposta pois não há menção à unicamp o...,0.176101,0
1,"Como ocorrem as atividades de cooperação, pesq...",nos termos do art 1° da deliberação consua0162...,mediante celebração de convênios contratos e i...,0.340426,0
2,Qual é o procedimento para a celebração de con...,na unicamp celebração de convênios contratos e...,procedimento para celebração de convênios cont...,0.407407,0
3,Qual é o sistema utilizado para a tramitação d...,os documentos essenciais estão elencados no ar...,processos administrativos eletrônicos,0.000000,0
4,O que é o Plano de Aplicação de Recursos?,plano de aplicação de recursos é documento que...,plano de aplicação de recursos é documento que...,0.341463,0
5,Quem pode ser executor de um convênio e quais ...,nos termos do art 18 §1° da deliberação consua...,os servidores ativos da unicamp de qualquer ca...,0.382353,0
6,Quem é a autoridade competente para assinatura...,como regra autoridade competente para assinatu...,as autoridades competentes para assinatura dos...,0.272727,0
7,Existe uma tramitação simplificada para aprova...,sim art 7º da deliberação consua0162022 estabe...,sim,0.009852,0
8,Quando se deve utilizar um Termo Aditivo a um ...,em razão da inexistência de única lei ou códig...,quando se deseja alterar os termos do convênio...,0.291971,0
9,É possível que o convênio preveja o pagamento ...,sim concessão de bolsas estímulo à inovação pe...,sim,0.010204,0


In [26]:
perg_resp_esperada_obtida[3]

{'pergunta': 'Qual é o sistema utilizado para a tramitação dos convênios?',
 'resposta_esperada': 'Os documentos essenciais estão elencados no art. 2º da Deliberação CONSU-A-016/2022. É importante que o Executor tenha em mente que estes são os documentos mínimos necessários, mas dependendo do objeto do convênio, outros documentos podem ser demandados. O processo do convênio deve se iniciar com o documento de apresentação da proposta elaborado por seu Executor, com a indicação dos motivos pelos quais apresenta a proposta e por que entende que sua celebração é conveniente e oportuna para a Universidade. A apresentação da proposta deve ser seguida dos demais documentos elencados no mencionado artigo 2º, além de outros documentos que forem pertinentes ao seu objeto. É importante que no sistema SIAD os documentos sejam incluídos separadamente e não num único arquivo. Sempre que o convênio fizer referência a um documento, este documento deverá ser incluído no processo, para que possa ser con

In [27]:
len(perg_resp_esperada_obtida)

50